In [19]:
import pandas as pd
import numpy as np
import os
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings('ignore')
flow = pd.read_csv('data/flow_train.csv')
trans = pd.read_csv('data/transition_train.csv')

cc_se = pd.Series(flow['city_code'].value_counts().index)
dc_se = pd.Series(flow['district_code'].value_counts().index)

flow['city_code'] = flow['city_code'].map(pd.Series(cc_se.index,index=cc_se.values))
flow['district_code'] = flow['district_code'].map(pd.Series(dc_se.index,index=dc_se.values))

for i in range(15):
    rs = []
    labels =['dwell', 'flow_in', 'flow_out']
    for label in labels:
        train = []
        test = []
        for va in flow[['city_code','district_code']].drop_duplicates().values:
            cc = va[0]
            dc = va[1]
            price = flow[(flow['city_code']==cc)&(flow['district_code']==dc)][label].values.tolist()
            k = len(price)
            diff = 30
            l = [cc,dc]
            l.extend(price[k-diff:k])
            l.append(np.nan)
            test.append(l)
            while k-15-diff>=0:
                l = [cc,dc]
                l.extend(price[k-1-diff:k-1])
                l.append(price[k-1])
                k = k-1
                train.append(l)

        test_x = pd.DataFrame(test)
        train_x = pd.DataFrame(train)
        del test_x[train_x.columns[-1]]
        train_y = train_x.pop(train_x.columns[-1])

        params_default_lgb = {
                'num_leaves':80, 
                'learning_rate':0.1, 
            'boosting':'gbdt',
            'min_child_samples':20,

            'bagging_fraction':1, 
            'bagging_freq':1,
            'feature_fraction':1, 
             'max_depth':-1,
            'reg_alpha':2,
            'reg_lambda':5, 
            'metric':'rmse',
            'objective':'regression'
        }
        NBR = 20000
        VBE = 1000
        ESR = 200
        def get_predict_result(k):
            df = pd.DataFrame()
            num = 0
            rmse = []
            skf = KFold(n_splits=5,random_state=2020,shuffle=True)
            for train_part_index,evals_index in skf.split(train_x,train_y):
                if k==1:
                    train_part = lgb.Dataset(train_x.loc[train_part_index],weight=train_weight.loc[train_part_index],label=train_y.loc[train_part_index])
                    evals = lgb.Dataset(train_x.loc[evals_index],weight=train_weight.loc[evals_index],label=train_y.loc[evals_index])
                elif k==0:
                    train_part = lgb.Dataset(train_x.loc[train_part_index],label=train_y.loc[train_part_index])
                    evals = lgb.Dataset(train_x.loc[evals_index],label=train_y.loc[evals_index])
                bst = lgb.train(params_default_lgb,train_part, 
                      num_boost_round=NBR, valid_sets=[train_part,evals], categorical_feature=[0,1],
                      valid_names=['train','evals'],early_stopping_rounds=ESR,
                         verbose_eval=False)
                num+=1
                valid_ypre = bst.predict(test_x)
                df['pred_'+str(num)]=valid_ypre
                rmse.append(bst.best_score['evals']['rmse'])
            return df,rmse
        df,rmse = get_predict_result(0)
        rs.append(np.array(rmse).mean())
        if label=='dwell':
            sub = test_x[[0,1]]
            sub[label] = df.mean(1)
        else:
            sub[label] = df.mean(1)

    print(rs)
    print(np.array(rs).mean())
    sub.columns = ['city_code','district_code', 'dwell', 'flow_in', 'flow_out']
    sub['date_dt'] = 20180302+i
    print(20180302+i)
    sub = sub[flow.columns]
    flow = pd.concat([flow,sub],axis=0,ignore_index=True)
    print(flow.shape)
    print('\n')
flow['city_code'] = flow['city_code'].map(cc_se)
flow['district_code'] = flow['district_code'].map(dc_se)
res = flow[flow['date_dt']>=20180302]
res.to_csv('result/QianYi_30.csv',index=False,header=None)
0.1293
只用到flow数据

[42.773995143781825, 84.94997640175379, 87.74666761327956]
71.82354638627173
20180302
(26950, 6)


[42.391649274145024, 83.90592714877542, 87.40569861554]
71.23442501282015
20180303
(27048, 6)


[43.50914385688486, 85.4148331176038, 88.5778211749592]
72.50059938314928
20180304
(27146, 6)


[41.349986086983854, 83.68108938430322, 86.77564672568307]
70.60224073232338
20180305
(27244, 6)


[42.15510345233334, 83.97172277813817, 87.85902826552376]
71.32861816533176
20180306
(27342, 6)


[42.61822990003054, 84.01863539170287, 86.63230757833027]
71.08972429002124
20180307
(27440, 6)


[42.01114858534301, 83.05932149017843, 84.66264487922409]
69.91103831824852
20180308
(27538, 6)


[41.578646674212365, 83.92700766285864, 86.2052415722072]
70.57029863642607
20180309
(27636, 6)


[42.302867994369045, 82.3779768863371, 86.28046252049947]
70.32043580040187
20180310
(27734, 6)


[42.806017924527346, 84.49964230185641, 88.50670060395203]
71.93745361011193
20180311
(27832, 6)


[41.71355104773364, 8

In [22]:
###写特殊的评分函数
import pandas as pd
import numpy as np
import os
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings('ignore')
flow = pd.read_csv('data/flow_train.csv')
trans = pd.read_csv('data/transition_train.csv')

cc_se = pd.Series(flow['city_code'].value_counts().index)
dc_se = pd.Series(flow['district_code'].value_counts().index)

flow['city_code'] = flow['city_code'].map(pd.Series(cc_se.index,index=cc_se.values))
flow['district_code'] = flow['district_code'].map(pd.Series(dc_se.index,index=dc_se.values))

for i in range(15):
    rs = []
    labels =['dwell', 'flow_in', 'flow_out']
    for label in labels:
        train = []
        test = []
        for va in flow[['city_code','district_code']].drop_duplicates().values:
            cc = va[0]
            dc = va[1]
            price = flow[(flow['city_code']==cc)&(flow['district_code']==dc)][label].values.tolist()
            k = len(price)
            diff = 30
            l = [cc,dc]
            l.extend(price[k-diff:k])
            l.append(np.nan)
            test.append(l)
            while k-15-diff>=0:
                l = [cc,dc]
                l.extend(price[k-1-diff:k-1])
                l.append(price[k-1])
                k = k-1
                train.append(l)

        test_x = pd.DataFrame(test)
        train_x = pd.DataFrame(train)
        del test_x[train_x.columns[-1]]
        train_y = train_x.pop(train_x.columns[-1])

        params_default_lgb = {
                'num_leaves':80, 
                'learning_rate':0.1, 
            'boosting':'gbdt',
            'min_child_samples':20,

            'bagging_fraction':1, 
            'bagging_freq':1,
            'feature_fraction':1, 
             'max_depth':-1,
            'reg_alpha':2,
            'reg_lambda':5, 
            'metric':'rmse',
            'objective':'regression'
        }
        NBR = 20000
        VBE = 1000
        ESR = 500
        def my_score(preds,data_vali):
            labels = data_vali.get_label()
            return 'rmsle',np.sqrt(mean_squared_error(np.log(preds+1),np.log(labels+1))),False
        def get_predict_result(k):
            df = pd.DataFrame()
            num = 0
            rmse = []
            skf = KFold(n_splits=5,random_state=2020,shuffle=True)
            for train_part_index,evals_index in skf.split(train_x,train_y):
                EVAL_RESULT = {}
                if k==1:
                    train_part = lgb.Dataset(train_x.loc[train_part_index],weight=train_weight.loc[train_part_index],label=train_y.loc[train_part_index])
                    evals = lgb.Dataset(train_x.loc[evals_index],weight=train_weight.loc[evals_index],label=train_y.loc[evals_index])
                elif k==0:
                    train_part = lgb.Dataset(train_x.loc[train_part_index],label=train_y.loc[train_part_index])
                    evals = lgb.Dataset(train_x.loc[evals_index],label=train_y.loc[evals_index])
                bst = lgb.train(params_default_lgb,train_part, feval = my_score,
                      num_boost_round=NBR, valid_sets=[train_part,evals], categorical_feature=[0,1],
                      valid_names=['train','evals'],early_stopping_rounds=ESR,evals_result=EVAL_RESULT,
                         verbose_eval=False)
                lst = EVAL_RESULT['evals']['rmsle']
                best_score = min(lst)
                best_iter = lst.index(best_score)+1
                valid_ypre = bst.predict(test_x,num_iteration = best_iter)
                num+=1
                df['pred_'+str(num)]=valid_ypre
                rmse.append(best_score)
            return df,rmse
        df,rmse = get_predict_result(0)
        rs.append(np.array(rmse).mean())
        if label=='dwell':
            sub = test_x[[0,1]]
            sub[label] = df.mean(1)
        else:
            sub[label] = df.mean(1)

    print(rs)
    print(np.array(rs).mean())
    sub.columns = ['city_code','district_code', 'dwell', 'flow_in', 'flow_out']
    sub['date_dt'] = 20180302+i
    print(20180302+i)
    sub = sub[flow.columns]
    flow = pd.concat([flow,sub],axis=0,ignore_index=True)
    print(flow.shape)
    print('\n')
flow['city_code'] = flow['city_code'].map(cc_se)
flow['district_code'] = flow['district_code'].map(dc_se)
res = flow[flow['date_dt']>=20180302]
res.to_csv('result/QianYi_30_RMSLE.csv',index=False,header=None)
##更换评分函数几乎没有区别
1293

[0.10692911831238475, 0.1480187085033961, 0.14650654773836985]
0.13381812485138359
20180302
(26950, 6)


[0.10619556642117076, 0.14729573244369673, 0.14587943524859154]
0.13312357803781968
20180303
(27048, 6)


[0.1053915946712302, 0.14696761302454658, 0.14512471227003246]
0.13249463998860309
20180304
(27146, 6)


[0.10567083126264923, 0.1461453124230704, 0.14522354256114237]
0.13234656208228734
20180305
(27244, 6)


[0.1058914427041191, 0.14681953467717662, 0.1451187616295106]
0.13260991300360211
20180306
(27342, 6)


[0.10506066395952192, 0.14549954339807064, 0.1440365668266641]
0.13153225806141888
20180307
(27440, 6)


[0.10525320715622395, 0.1459723456938515, 0.1449527559123776]
0.13205943625415104
20180308
(27538, 6)


[0.10481150788752762, 0.14573067552930924, 0.1440287265276819]
0.13152363664817293
20180309
(27636, 6)


[0.10441522056957411, 0.14579135219834083, 0.14416557475841782]
0.1314573825087776
20180310
(27734, 6)


[0.10525377970008667, 0.14530152809435393, 0.14384588743

In [3]:
import pandas as pd
import numpy as np
import os
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings('ignore')
flow = pd.read_csv('data/flow_train.csv')
trans = pd.read_csv('data/transition_train.csv')

cc_se = pd.Series(flow['city_code'].value_counts().index)
dc_se = pd.Series(flow['district_code'].value_counts().index)

flow['city_code'] = flow['city_code'].map(pd.Series(cc_se.index,index=cc_se.values))
flow['district_code'] = flow['district_code'].map(pd.Series(dc_se.index,index=dc_se.values))

for i in range(15):
    rs = []
    labels =['dwell', 'flow_in', 'flow_out']
    for label in labels:
        train = []
        test = []
        for va in flow[['city_code','district_code']].drop_duplicates().values:
            cc = va[0]
            dc = va[1]
            price = flow[(flow['city_code']==cc)&(flow['district_code']==dc)][label].values.tolist()
            k = len(price)
            diff = 30
            l = [cc,dc]
            l.extend(price[k-diff:k])
            l.append(np.nan)
            test.append(l)
            while k-15-diff>=0:
                l = [cc,dc]
                l.extend(price[k-1-diff:k-1])
                l.append(price[k-1])
                k = k-1
                train.append(l)

        test_x = pd.DataFrame(test)
        train_x = pd.DataFrame(train)
        del test_x[train_x.columns[-1]]
        train_y = train_x.pop(train_x.columns[-1])
        
        test_x['week_day'] = 0
        train_x['week_day'] = (train_x.index.values+1)%7
        def preprocess(dt):
            for i in range(7):
                dt['week_day_'+str(i)] =(dt['week_day']==i).astype(int)
            del dt['week_day']
            return dt
        train_x = preprocess(train_x)
        test_x = preprocess(test_x)
        
        params_default_lgb = {
                'num_leaves':80, 
                'learning_rate':0.1, 
            'boosting':'gbdt',
            'min_child_samples':20,

            'bagging_fraction':1, 
            'bagging_freq':1,
            'feature_fraction':1, 
             'max_depth':-1,
            'reg_alpha':2,
            'reg_lambda':5, 
            'metric':'rmse',
            'objective':'regression'
        }
        NBR = 20000
        VBE = 1000
        ESR = 200
        def get_predict_result(k):
            df = pd.DataFrame()
            num = 0
            rmse = []
            skf = KFold(n_splits=5,random_state=2020,shuffle=True)
            for train_part_index,evals_index in skf.split(train_x,train_y):
                if k==1:
                    train_part = lgb.Dataset(train_x.loc[train_part_index],weight=train_weight.loc[train_part_index],label=train_y.loc[train_part_index])
                    evals = lgb.Dataset(train_x.loc[evals_index],weight=train_weight.loc[evals_index],label=train_y.loc[evals_index])
                elif k==0:
                    train_part = lgb.Dataset(train_x.loc[train_part_index],label=train_y.loc[train_part_index])
                    evals = lgb.Dataset(train_x.loc[evals_index],label=train_y.loc[evals_index])
                bst = lgb.train(params_default_lgb,train_part, 
                      num_boost_round=NBR, valid_sets=[train_part,evals], categorical_feature=[0,1],
                      valid_names=['train','evals'],early_stopping_rounds=ESR,
                         verbose_eval=False)
                num+=1
                valid_ypre = bst.predict(test_x)
                df['pred_'+str(num)]=valid_ypre
                rmse.append(bst.best_score['evals']['rmse'])
            return df,rmse
        df,rmse = get_predict_result(0)
        rs.append(np.array(rmse).mean())
        if label=='dwell':
            sub = test_x[[0,1]]
            sub[label] = df.mean(1)
        else:
            sub[label] = df.mean(1)

    print(rs)
    print(np.array(rs).mean())
    sub.columns = ['city_code','district_code', 'dwell', 'flow_in', 'flow_out']
    sub['date_dt'] = 20180302+i
    print(20180302+i)
    sub = sub[flow.columns]
    flow = pd.concat([flow,sub],axis=0,ignore_index=True)
    print(flow.shape)
    print('\n')
flow['city_code'] = flow['city_code'].map(cc_se)
flow['district_code'] = flow['district_code'].map(dc_se)
res = flow[flow['date_dt']>=20180302]
res.to_csv('result/QianYi_30_Week.csv',index=False,header=None)
1336

[42.33505707367633, 84.36296860338554, 87.1267164205249]
71.27491403252893
20180302
(26950, 6)


[41.10180883382431, 81.93088600985118, 85.43313336903176]
69.48860940423576
20180303
(27048, 6)


[42.909948947209884, 84.81597140572971, 87.5965103309589]
71.77414356129951
20180304
(27146, 6)


[40.8733697987862, 82.96288442393374, 86.42877771561594]
70.0883439794453
20180305
(27244, 6)


[41.8378123489881, 82.72226421020764, 87.40708576875541]
70.65572077598371
20180306
(27342, 6)


[42.56583817261663, 82.87107965886555, 85.94380908344085]
70.46024230497436
20180307
(27440, 6)


[41.37499687591405, 81.20588530371012, 84.35580828699864]
68.9788968222076
20180308
(27538, 6)


[41.28785042641101, 83.41649668590465, 85.43006431943415]
70.04480381058328
20180309
(27636, 6)


[40.5677486303062, 81.05868769258879, 84.5171431283732]
68.71452648375606
20180310
(27734, 6)


[42.20353549707947, 83.51484659564271, 87.01732956289558]
70.91190388520592
20180311
(27832, 6)


[40.90288256912637, 80.8121

In [8]:
import pandas as pd
import numpy as np
import os
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings('ignore')
flow = pd.read_csv('data/flow_train.csv')
trans = pd.read_csv('data/transition_train.csv')

cc_se = pd.Series(flow['city_code'].value_counts().index)
dc_se = pd.Series(flow['district_code'].value_counts().index)

flow['city_code'] = flow['city_code'].map(pd.Series(cc_se.index,index=cc_se.values))
flow['district_code'] = flow['district_code'].map(pd.Series(dc_se.index,index=dc_se.values))

for i in range(15):
    rs = []
    labels =['dwell', 'flow_in', 'flow_out']
    for label in labels:
        train = []
        test = []
        for va in flow[['city_code','district_code']].drop_duplicates().values:
            cc = va[0]
            dc = va[1]
            price = flow[(flow['city_code']==cc)&(flow['district_code']==dc)][label].values.tolist()
            k = len(price)
            diff = 30
            l = [cc,dc]
            l.extend(price[k-diff:k])
            l.append(np.nan)
            test.append(l)
            while k-15-diff>=0:
                l = [cc,dc]
                l.extend(price[k-1-diff:k-1])
                l.append(price[k-1])
                k = k-1
                train.append(l)

        test_x = pd.DataFrame(test)
        train_x = pd.DataFrame(train)
        del test_x[train_x.columns[-1]]
        train_y = train_x.pop(train_x.columns[-1])
        def getFeature(dt):
            col = dt.columns.tolist()
            for i in range(1,5):
                co1 = col[-i]
                co2 = col[-i-1]
                dt[str(co1)+'-'+str(co2)] = dt[co1]-dt[co2]
            return dt
        train_x = getFeature(train_x)
        test_x = getFeature(test_x)
        params_default_lgb = {
                'num_leaves':80, 
                'learning_rate':0.1, 
            'boosting':'gbdt',
            'min_child_samples':20,

            'bagging_fraction':1, 
            'bagging_freq':1,
            'feature_fraction':1, 
             'max_depth':-1,
            'reg_alpha':2,
            'reg_lambda':5, 
            'metric':'rmse',
            'objective':'regression'
        }
        NBR = 20000
        VBE = 1000
        ESR = 200
        def get_predict_result(k):
            df = pd.DataFrame()
            num = 0
            rmse = []
            skf = KFold(n_splits=5,random_state=2020,shuffle=True)
            for train_part_index,evals_index in skf.split(train_x,train_y):
                if k==1:
                    train_part = lgb.Dataset(train_x.loc[train_part_index],weight=train_weight.loc[train_part_index],label=train_y.loc[train_part_index])
                    evals = lgb.Dataset(train_x.loc[evals_index],weight=train_weight.loc[evals_index],label=train_y.loc[evals_index])
                elif k==0:
                    train_part = lgb.Dataset(train_x.loc[train_part_index],label=train_y.loc[train_part_index])
                    evals = lgb.Dataset(train_x.loc[evals_index],label=train_y.loc[evals_index])
                bst = lgb.train(params_default_lgb,train_part, 
                      num_boost_round=NBR, valid_sets=[train_part,evals], categorical_feature=[0,1],
                      valid_names=['train','evals'],early_stopping_rounds=ESR,
                         verbose_eval=False)
                num+=1
                valid_ypre = bst.predict(test_x)
                df['pred_'+str(num)]=valid_ypre
                rmse.append(bst.best_score['evals']['rmse'])
            return df,rmse
        df,rmse = get_predict_result(0)
        rs.append(np.array(rmse).mean())
        if label=='dwell':
            sub = test_x[[0,1]]
            sub[label] = df.mean(1)
        else:
            sub[label] = df.mean(1)

    print(rs)
    print(np.array(rs).mean())
    sub.columns = ['city_code','district_code', 'dwell', 'flow_in', 'flow_out']
    sub['date_dt'] = 20180302+i
    print(20180302+i)
    sub = sub[flow.columns]
    flow = pd.concat([flow,sub],axis=0,ignore_index=True)
    print(flow.shape)
    print('\n')
flow['city_code'] = flow['city_code'].map(cc_se)
flow['district_code'] = flow['district_code'].map(dc_se)
res = flow[flow['date_dt']>=20180302]
res.to_csv('result/QianYi_30_gap.csv',index=False,header=None)
1339

[40.92936566190821, 82.40624287289674, 83.96722061613461]
69.10094305031318
20180302
(26950, 6)


[40.9102531664693, 81.83374113552809, 83.94443786182028]
68.89614405460588
20180303
(27048, 6)


[41.72134937390068, 83.08066754697956, 85.56407151080627]
70.12202947722884
20180304
(27146, 6)


[39.87603698337692, 80.94226990733696, 83.79960341634265]
68.20597010235218
20180305
(27244, 6)


[40.42335589102329, 80.96760172155571, 84.50548591816525]
68.63214784358142
20180306
(27342, 6)


[40.95068137447059, 80.58776074335609, 83.19629912908707]
68.24491374897126
20180307
(27440, 6)


[39.678288005871025, 79.63178288344511, 82.07008036695791]
67.12671708542469
20180308
(27538, 6)


[40.169801323185894, 80.54865309124536, 82.43760165174908]
67.71868535539345
20180309
(27636, 6)


[39.802635714670046, 80.24207746415966, 83.7552662742783]
67.93332648436933
20180310
(27734, 6)


[41.155488863144306, 81.89988742686603, 84.62317088715668]
69.226182392389
20180311
(27832, 6)


[39.7702009676622, 7

In [9]:
import pandas as pd
import numpy as np
import os
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings('ignore')
flow = pd.read_csv('data/flow_train.csv')
trans = pd.read_csv('data/transition_train.csv')

cc_se = pd.Series(flow['city_code'].value_counts().index)
dc_se = pd.Series(flow['district_code'].value_counts().index)

flow['city_code'] = flow['city_code'].map(pd.Series(cc_se.index,index=cc_se.values))
flow['district_code'] = flow['district_code'].map(pd.Series(dc_se.index,index=dc_se.values))

for i in range(15):
    rs = []
    labels =['dwell', 'flow_in', 'flow_out']
    for label in labels:
        train = []
        test = []
        for va in flow[['city_code','district_code']].drop_duplicates().values:
            cc = va[0]
            dc = va[1]
            price = flow[(flow['city_code']==cc)&(flow['district_code']==dc)][label].values.tolist()
            k = len(price)
            diff = 30
            l = [cc,dc]
            l.extend(price[k-diff:k])
            l.append(np.nan)
            test.append(l)
            while k-15-diff>=0:
                l = [cc,dc]
                l.extend(price[k-1-diff:k-1])
                l.append(price[k-1])
                k = k-1
                train.append(l)

        test_x = pd.DataFrame(test)
        train_x = pd.DataFrame(train)
        del test_x[train_x.columns[-1]]
        train_y = train_x.pop(train_x.columns[-1])
        def getFeature(dt):
            col = dt.columns.tolist()
            for i in range(1,15):
                co1 = col[-i]
                co2 = col[-i-1]
                dt[str(co1)+'-'+str(co2)] = dt[co1]-dt[co2]
            return dt
        train_x = getFeature(train_x)
        test_x = getFeature(test_x)
        params_default_lgb = {
                'num_leaves':80, 
                'learning_rate':0.1, 
            'boosting':'gbdt',
            'min_child_samples':20,

            'bagging_fraction':1, 
            'bagging_freq':1,
            'feature_fraction':1, 
             'max_depth':-1,
            'reg_alpha':2,
            'reg_lambda':5, 
            'metric':'rmse',
            'objective':'regression'
        }
        NBR = 20000
        VBE = 1000
        ESR = 200
        def get_predict_result(k):
            df = pd.DataFrame()
            num = 0
            rmse = []
            skf = KFold(n_splits=5,random_state=2020,shuffle=True)
            for train_part_index,evals_index in skf.split(train_x,train_y):
                if k==1:
                    train_part = lgb.Dataset(train_x.loc[train_part_index],weight=train_weight.loc[train_part_index],label=train_y.loc[train_part_index])
                    evals = lgb.Dataset(train_x.loc[evals_index],weight=train_weight.loc[evals_index],label=train_y.loc[evals_index])
                elif k==0:
                    train_part = lgb.Dataset(train_x.loc[train_part_index],label=train_y.loc[train_part_index])
                    evals = lgb.Dataset(train_x.loc[evals_index],label=train_y.loc[evals_index])
                bst = lgb.train(params_default_lgb,train_part, 
                      num_boost_round=NBR, valid_sets=[train_part,evals], categorical_feature=[0,1],
                      valid_names=['train','evals'],early_stopping_rounds=ESR,
                         verbose_eval=False)
                num+=1
                valid_ypre = bst.predict(test_x)
                df['pred_'+str(num)]=valid_ypre
                rmse.append(bst.best_score['evals']['rmse'])
            return df,rmse
        df,rmse = get_predict_result(0)
        rs.append(np.array(rmse).mean())
        if label=='dwell':
            sub = test_x[[0,1]]
            sub[label] = df.mean(1)
        else:
            sub[label] = df.mean(1)

    print(rs)
    print(np.array(rs).mean())
    sub.columns = ['city_code','district_code', 'dwell', 'flow_in', 'flow_out']
    sub['date_dt'] = 20180302+i
    print(20180302+i)
    sub = sub[flow.columns]
    flow = pd.concat([flow,sub],axis=0,ignore_index=True)
    print(flow.shape)
    print('\n')
flow['city_code'] = flow['city_code'].map(cc_se)
flow['district_code'] = flow['district_code'].map(dc_se)
res = flow[flow['date_dt']>=20180302]
res.to_csv('result/QianYi_30_gap_15.csv',index=False,header=None)

[39.0306453859712, 77.51183805366145, 77.45152953787368]
64.66467099250211
20180302
(26950, 6)


[38.37923233780531, 75.16307467752311, 76.42925299789667]
63.323853337741696
20180303
(27048, 6)


[38.55056066938981, 76.93507147209621, 78.50188338211638]
64.66250517453413
20180304
(27146, 6)


[37.06174403605, 76.2428137526013, 76.98778673741411]
63.43078150868846
20180305
(27244, 6)


[38.04382567154083, 75.649043455882, 77.49228527395135]
63.72838480045806
20180306
(27342, 6)


[38.4701164305625, 74.32424421353004, 74.82085480036105]
62.53840514815119
20180307
(27440, 6)


[38.15102661429068, 73.10122082168809, 75.47845187859097]
62.24356643818991
20180308
(27538, 6)


[37.46452804169385, 75.16355560155813, 76.40993035457981]
63.0126713326106
20180309
(27636, 6)


[36.468715659669634, 73.5303472141766, 76.0941011070036]
62.03105466028328
20180310
(27734, 6)


[37.16952015552567, 76.17070877970427, 77.9494587386325]
63.76322922462081
20180311
(27832, 6)


[36.94219237488674, 72.1469642

In [10]:
import pandas as pd
import numpy as np
import os
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings('ignore')
flow = pd.read_csv('data/flow_train.csv')
trans = pd.read_csv('data/transition_train.csv')

cc_se = pd.Series(flow['city_code'].value_counts().index)
dc_se = pd.Series(flow['district_code'].value_counts().index)

flow['city_code'] = flow['city_code'].map(pd.Series(cc_se.index,index=cc_se.values))
flow['district_code'] = flow['district_code'].map(pd.Series(dc_se.index,index=dc_se.values))

for i in range(15):
    rs = []
    labels =['dwell', 'flow_in', 'flow_out']
    for label in labels:
        train = []
        test = []
        for va in flow[['city_code','district_code']].drop_duplicates().values:
            cc = va[0]
            dc = va[1]
            price = flow[(flow['city_code']==cc)&(flow['district_code']==dc)][label].values.tolist()
            k = len(price)
            diff = 30
            l = [cc,dc]
            l.extend(price[k-diff:k])
            l.append(np.nan)
            test.append(l)
            while k-15-diff>=0:
                l = [cc,dc]
                l.extend(price[k-1-diff:k-1])
                l.append(price[k-1])
                k = k-1
                train.append(l)

        test_x = pd.DataFrame(test)
        train_x = pd.DataFrame(train)
        del test_x[train_x.columns[-1]]
        train_y = train_x.pop(train_x.columns[-1])
        def getFeature(dt):
            col = dt.columns.tolist()
            for i in range(1,30):
                co1 = col[-i]
                co2 = col[-i-1]
                dt[str(co1)+'-'+str(co2)] = dt[co1]-dt[co2]
            return dt
        train_x = getFeature(train_x)
        test_x = getFeature(test_x)
        params_default_lgb = {
                'num_leaves':80, 
                'learning_rate':0.1, 
            'boosting':'gbdt',
            'min_child_samples':20,

            'bagging_fraction':1, 
            'bagging_freq':1,
            'feature_fraction':1, 
             'max_depth':-1,
            'reg_alpha':2,
            'reg_lambda':5, 
            'metric':'rmse',
            'objective':'regression'
        }
        NBR = 20000
        VBE = 1000
        ESR = 200
        def get_predict_result(k):
            df = pd.DataFrame()
            num = 0
            rmse = []
            skf = KFold(n_splits=5,random_state=2020,shuffle=True)
            for train_part_index,evals_index in skf.split(train_x,train_y):
                if k==1:
                    train_part = lgb.Dataset(train_x.loc[train_part_index],weight=train_weight.loc[train_part_index],label=train_y.loc[train_part_index])
                    evals = lgb.Dataset(train_x.loc[evals_index],weight=train_weight.loc[evals_index],label=train_y.loc[evals_index])
                elif k==0:
                    train_part = lgb.Dataset(train_x.loc[train_part_index],label=train_y.loc[train_part_index])
                    evals = lgb.Dataset(train_x.loc[evals_index],label=train_y.loc[evals_index])
                bst = lgb.train(params_default_lgb,train_part, 
                      num_boost_round=NBR, valid_sets=[train_part,evals], categorical_feature=[0,1],
                      valid_names=['train','evals'],early_stopping_rounds=ESR,
                         verbose_eval=False)
                num+=1
                valid_ypre = bst.predict(test_x)
                df['pred_'+str(num)]=valid_ypre
                rmse.append(bst.best_score['evals']['rmse'])
            return df,rmse
        df,rmse = get_predict_result(0)
        rs.append(np.array(rmse).mean())
        if label=='dwell':
            sub = test_x[[0,1]]
            sub[label] = df.mean(1)
        else:
            sub[label] = df.mean(1)

    print(rs)
    print(np.array(rs).mean())
    sub.columns = ['city_code','district_code', 'dwell', 'flow_in', 'flow_out']
    sub['date_dt'] = 20180302+i
    print(20180302+i)
    sub = sub[flow.columns]
    flow = pd.concat([flow,sub],axis=0,ignore_index=True)
    print(flow.shape)
    print('\n')
flow['city_code'] = flow['city_code'].map(cc_se)
flow['district_code'] = flow['district_code'].map(dc_se)
res = flow[flow['date_dt']>=20180302]
res.to_csv('result/QianYi_30_gap_30.csv',index=False,header=None)

[35.572009699986346, 70.06044080102653, 71.68592211670696]
59.10612420590661
20180302
(26950, 6)


[35.09905032931677, 67.26895958777729, 70.31305719417549]
57.56035570375652
20180303
(27048, 6)


[35.01337883856839, 69.40450134738796, 71.13677642147522]
58.51821886914386
20180304
(27146, 6)


[34.41654237534212, 67.96449351881651, 70.33182112992026]
57.57095234135963
20180305
(27244, 6)


[34.91084368980053, 68.81616520420584, 72.3240358162644]
58.683681570090265
20180306
(27342, 6)


[35.53659749914308, 67.83833062163271, 68.5638692617105]
57.31293246082876
20180307
(27440, 6)


[34.74727698961436, 66.85926186530085, 68.75101076679773]
56.78584987390432
20180308
(27538, 6)


[34.47782858199112, 67.15382527290384, 69.61996090194984]
57.08387158561493
20180309
(27636, 6)


[34.00995465294136, 67.87302389198929, 68.52205359724073]
56.80167738072379
20180310
(27734, 6)


[35.11033737026689, 69.04616341749323, 70.88022828012302]
58.34557635596104
20180311
(27832, 6)


[33.9921589132569, 6